# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [ ]:
# Write your code below.
%load_ext dotenv
%dotenv

In [2]:
import dask.dataframe as dd

c:\Users\Kirby\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [5]:
import os
from glob import glob

# Write your code below.

print(os.getenv('PRICE_DATA'))

PRICE_DATA = os.getenv('PRICE_DATA')

parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
dd_px = dd.read_parquet(parquet_files).set_index("Ticker")


../../05_src/data/prices/


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [12]:
# Write your code below.
shift_dd = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(close_lag = x['Close'].shift(1))
)

dd_feat = shift_dd.assign(
    returns = lambda x: x['Close']/x['close_lag'] - 1,
    hi_lo_range = lambda x: x['High'] - x['Low']
)

dd_feat

C:\Users\Kirby\AppData\Local\Temp\ipykernel_7028\2108636153.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  shift_dd = dd_px.groupby('Ticker', group_keys=False).apply(


,Date,Close,High,Low,Open,Volume,Year,close_lag,returns,hi_lo_range
npartitions=64,,,,,,,,,,
AAPL,datetime64[ns],float64,float64,float64,float64,float64,int32,float64,float64,float64
ACN,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
ZBRA,...,...,...,...,...,...,...,...,...,...
ZBRA,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [30]:
# Write your code below.
df = dd_feat.compute()

df['avg_10_day_return'] = (
    df.groupby('Ticker')['returns'].rolling(10).mean().sort_index(level=1).values
    )


df[df.index == 'MSFT'].iloc[0:11]

,Date,Close,High,Low,Open,Volume,Year,close_lag,returns,hi_lo_range,avg_10_day_return
Ticker,,,,,,,,,,,
MSFT,2000-01-03,35.726730,36.358892,34.328311,35.975764,53228400.0,2000,NaN,NaN,2.030581,NaN
MSFT,2000-01-04,34.519863,35.899125,34.404925,34.807209,54119000.0,2000,35.726730,-0.033781,1.494201,NaN
MSFT,2000-01-05,34.883831,35.669244,33.523726,34.060105,64059600.0,2000,34.519863,0.010544,2.145518,NaN
MSFT,2000-01-06,33.715305,34.903004,33.217238,34.385780,54976600.0,2000,34.883831,-0.033498,1.685765,NaN
MSFT,2000-01-07,34.155907,34.404940,32.891583,33.293868,62013600.0,2000,33.715305,0.013068,1.513358,NaN
MSFT,2000-01-10,34.404930,34.845528,34.136740,34.768902,44963600.0,2000,34.155907,0.007291,0.708788,NaN
MSFT,2000-01-11,33.523746,35.017948,33.313026,34.175065,46743600.0,2000,34.404930,-0.025612,1.704922,NaN
MSFT,2000-01-12,32.431812,33.370477,32.010371,33.255538,66532400.0,2000,33.523746,-0.032572,1.360106,NaN
MSFT,2000-01-13,33.044838,33.293872,31.110039,31.991234,83144000.0,2000,32.431812,0.018902,2.183833,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

# Was it necessary to convert to pandas to calculate the moving average return?

No, we could have done it while using a Dask Dataframe

# Would it have been better to do it in Dask? Why?

No - since the dataset is relatively small - < 40 MB - pandas can just read the entire dataset in memory and do the calcations there.
With Dask, we'd have to pick out 10 rows per ticker (the index) and calculate the mean. Dask is efficient when maniulating data based off the index, not the features. 

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.